# Worksheet 08

Name: Xudong Wang   
UID: U91936499

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.39760234389257, 5.778371151108986, 4.6226350396958305, 5.652242933276219, 3.7737001788782023, 4.98430881558656, 5.154687380324923, 5.374983000840603, 3.3392812909295397, 3.714035394616637]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[9.595131553543577, 9.613199063196756, 7.749212618320668, 8.645413260334383, 8.030328296225603, 5.856215174259658, 6.99064526032186, 9.117975706978893, 7.914939563442569, 8.471182942434092]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[3.714035394616637, 8.471182942434092, 3.3392812909295397, 5.374983000840603, 5.154687380324923, 7.914939563442569, 4.98430881558656, 3.7737001788782023, 9.117975706978893, 6.99064526032186]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Mean, variance of the weight in each species, and mixture weights.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [7]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[8.471182942434092, 7.914939563442569, 9.117975706978893, 6.99064526032186]
[3.714035394616637, 3.3392812909295397, 5.374983000840603, 5.154687380324923, 4.98430881558656, 3.7737001788782023]
P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 8.123685868294354,  mean_2 = 4.390166010196077
var_1 = 0.7805002464687244,  var_2 = 0.8009270178296815


They are close to the real one

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [8]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # pdf_i(0) is p(p|s_0)

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0]*prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1]*prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  3.714035394616637
probability of observing that point if it came from cluster 0 =  5.986972603826898e-08
probability of observing that point if it came from cluster 1 =  0.3487927810540888
point =  8.471182942434092
probability of observing that point if it came from cluster 0 =  0.4629064461976769
probability of observing that point if it came from cluster 1 =  1.1470301049561246e-06
point =  3.3392812909295397
probability of observing that point if it came from cluster 0 =  3.5402025590253708e-09
probability of observing that point if it came from cluster 1 =  0.21061209476997902
point =  5.374983000840603
probability of observing that point if it came from cluster 0 =  0.0010360166009982057
probability of observing that point if it came from cluster 1 =  0.2338894393928293
point =  5.154687380324923
probability of observing that point if it came from cluster 0 =  0.0003684494077843333
probability of observing that point if it came from cluster 1 =  0.31583613952664435
point

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [10]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum(x[0] * (x[1] - mean[0])**2 for x in zip(prob_s0_x, data)) / sum(prob_s0_x) , sum(x[0] * (x[1] - mean[1])**2 for x in zip(prob_s1_x, data)) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_c[0]) + ",  P(S_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.3982810034052061,  P(S_2) = 0.6017189965947939
mean_1 = 8.126860756342621,  mean_2 = 4.398730490217925
var_1 = 0.6129911525672916,  var_2 = 0.6629193353372256


The different are not quite big

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [16]:
mean_updated = mean
var_updated = var
prob_s_updated = prob_c

prob_s0_x_updated = [] # P(S_0 | X_i)
prob_s1_x_updated = [] # P(S_1 | X_i)
prob_x_updated = [] # P(X_i)

for p in data:
    pdf_i_updated = []
    for j in range(k):
        # P(X_i | S_j)
        pdf_i_updated.append(norm.pdf(p, mean_updated[j], var_updated[j]))
    #  P(X_i)
    prob_x_updated = prob_s_updated[0] * pdf_i_updated[0] + prob_s_updated[1] * pdf_i_updated[1]
    # P(S_j | X_i)
    prob_s0_x_updated.append(pdf_i_updated[0] * prob_s_updated[0] / prob_x_updated)
    prob_s1_x_updated.append(pdf_i_updated[1] * prob_s_updated[1] / prob_x_updated)

for i, p in enumerate(data):
    print(f" Data points{p}: P(S_0 | X_i) = {prob_s0_x_updated[i]}, P(S_1 | X_i) = {prob_s1_x_updated[i]}")

 Data points3.714035394616637: P(S_0 | X_i) = 6.810249263722238e-12, P(S_1 | X_i) = 0.9999999999931898
 Data points8.471182942434092: P(S_0 | X_i) = 0.999999989557944, P(S_1 | X_i) = 1.0442056028090372e-08
 Data points3.3392812909295397: P(S_0 | X_i) = 1.4575669903330363e-13, P(S_1 | X_i) = 0.9999999999998542
 Data points5.374983000840603: P(S_0 | X_i) = 8.900696232528017e-05, P(S_1 | X_i) = 0.9999109930376747
 Data points5.154687380324923: P(S_0 | X_i) = 1.076940112900813e-05, P(S_1 | X_i) = 0.999989230598871
 Data points7.914939563442569: P(S_0 | X_i) = 0.9999988466025636, P(S_1 | X_i) = 1.1533974365093866e-06
 Data points4.98430881558656: P(S_0 | X_i) = 2.075857227085457e-06, P(S_1 | X_i) = 0.999997924142773
 Data points3.7737001788782023: P(S_0 | X_i) = 1.2496808736201854e-11, P(S_1 | X_i) = 0.9999999999875032
 Data points9.117975706978893: P(S_0 | X_i) = 0.9999999999489296, P(S_1 | X_i) = 5.1070429846187426e-11
 Data points6.99064526032186: P(S_0 | X_i) = 0.9962836502664354, P(S_1

I didn't see any regular change

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [ ]:
hard_labels = [0 if s0 > s1 else 1 for s0, s1 in zip(prob_s0_x, prob_s1_x)]
